In [1]:
from netCDF4 import Dataset
import numpy as np
import glob
import os
import pandas as pd
import wrf

import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
import cartopy.feature as cfeature
from cartopy import config
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature

from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [89]:
wrfile_d02 = Dataset('/media/lucas/ce7c746b-5271-4ebe-a443-a00fe8fb43f4/wrf/WRF/run/wrfout_d02_2018-05-23_13:00:00')
torres = pd.read_csv('/home/lucas/dados_vento/lat_lon_towers.txt', usecols=[1,2]).values

In [5]:
u10 = wrf.getvar(wrfile_d02, 'U10')
v10 = wrf.getvar(wrfile_d02, 'V10')
wind10 = np.sqrt(u10*u10+v10*v10)

In [75]:
wspeed = wrf.getvar(wrfile_d02,'wspd_wdir')[0]
wdir = wrf.getvar(wrfile_d02,'wspd_wdir')[1]
z = wrf.getvar(wrfile_d02, 'z', units='m')
slp = wrf.getvar(wrfile_d02,'slp')
p = wrf.getvar(wrfile_d02,'pressure')
lon = wrf.getvar(wrfile_d02,'lon')
lat = wrf.getvar(wrfile_d02,'lat')

In [76]:
slp.shape,  wspeed[0,:,:].shape, z.shape

((30, 33), (30, 33), (34, 30, 33))

In [77]:
wrf.interplevel(wspeed, z, [200])

<xarray.DataArray 'wspd_wdir_interp' (south_north: 30, west_east: 33)>
array([[      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
        0.2578142,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,

In [91]:
n=0
coordenada=[]
wind=[]
while n<3:
    abslat = np.abs(lat-torres[n,0])
    abslon = np.abs(lon-torres[n,1])
    c = np.maximum(abslon,abslat)
    x, y = np.where(c == np.min(c))
    wind_ = wspeed[0,x,y]
    wind.append(wind_)
    n+=1

In [82]:
del(torres['Unnamed: 0'])

In [116]:
float(wind[0].values), float(wind[1].values), float(wind[2].values)

(1.731642723083496, 1.94505774974823, 2.032102108001709)

In [112]:
df = pd.read_csv('ecc1_.csv')
df2 = pd.read_csv('ecc2_.csv')
df3 = pd.read_csv('ecc3_.csv')

In [117]:
float(df[df.data=='2018-05-23 13:00:00']['vi_med']), \
float(df2[df2.data=='2018-05-23 13:00:00']['vi_med']), \
float(df3[df3.data=='2018-05-23 13:00:00']['vi_med'])

(1.55, 1.2, 0.64)

In [105]:
df

,data,hora,erro,hPa,tp,ur,vs_med,vs_max,vs_min,vs_dev,...,vi_med,vi_max,vi_min,vi_dev,di_med,di_dev,vm_med,vm_max,vm_min,vm_dev
0,2018-05-23 00:00:00,0,0,980,11.6,61.5,4.5,4.89,4.24,0.17,...,4.86,5.17,4.57,0.14,187,1,4.53,4.95,4.21,0.17
1,2018-05-23 00:10:00,1000,0,980,11.6,61.5,5.12,5.39,4.75,0.12,...,5.17,5.49,4.94,0.09,182,1,4.46,4.99,4.12,0.22
2,2018-05-23 00:20:00,2000,0,980,11.6,60.8,5.2,5.44,5.03,0.06,...,5.28,5.44,5.03,0.08,182,0,4.4,4.86,4.21,0.1
3,2018-05-23 00:30:00,3000,0,980,11.6,60.8,5.04,5.21,4.84,0.06,...,4.93,5.26,4.71,0.14,183,0,4.79,5.13,4.44,0.16
4,2018-05-23 00:40:00,4000,0,980,11.6,59.2,4.92,5.12,4.75,0.06,...,4.78,4.94,4.62,0.06,184,1,4.96,5.13,4.81,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,2018-12-24 23:10:00,231000,0,974,14.1,69.7,12.36,16.19,8.3,1.33,...,10.06,14.5,6.14,1.54,158,6,9.5,15.27,5.78,1.84
1580,2018-12-24 23:20:00,232000,0,974,14.1,64.7,16.57,20.21,10.05,1.68,...,13.1,18.18,6.96,1.82,158,5,12.04,16.98,7.85,1.72
1581,2018-12-24 23:30:00,233000,0,974,13.9,61.6,17.63,19.93,14.9,1.03,...,14.65,18.46,9.81,1.54,160,4,12.91,18.09,9.6,1.62
1582,2018-12-24 23:40:00,234000,0,975,13.6,61.3,14.49,18.41,11.12,1.55,...,11.77,16.11,8.02,1.67,157,4,10.82,17.39,7.25,2
